In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Problem Statement: predict a system’s probability of getting infected by various families of malware, based on different properties of that system. The telemetry data containing these properties and the system infections was generated by threat reports collected by system's antivirus software.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from category_encoders import TargetEncoder, CountEncoder

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold


### Loading Dataset

In [ ]:
train_data = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")
test_data = pd.read_csv("/kaggle/input/System-Threat-Forecaster/test.csv")
submission_data = pd.read_csv("/kaggle/input/System-Threat-Forecaster/sample_submission.csv")

## Exploratory Data Analysis

In [ ]:
print("\nTrain Data Info:")
train_data.info()

In [ ]:
print("\nTest Data Info:")
test_data.info()

In [ ]:
train_data.shape,test_data.shape

### Check for missing values

In [ ]:
# Check for missing values
missing_values = train_data.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
print("\nMissing Values in Train Data:")
print(missing_values)



In [ ]:
# Check for missing values
missing_values = test_data.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
print("\nMissing Values in Train Data:")
print(missing_values)

In [ ]:
train_data.head

In [ ]:
# Summary statistics of numerical features
print("\nNumerical Feature Summary:")
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.max_rows", None)
print(train_data.describe())

In [ ]:
# Check unique values in each column
unique_values = train_data.nunique()
unique_values = unique_values[unique_values > 0].sort_values(ascending=False)
print("\nUnique Values per Column in Train Data:")
print(unique_values)

In [ ]:
# Analyze target distribution
plt.figure(figsize=(6,4))
sns.countplot(x=train_data['target'])
plt.title("Target Distribution")
plt.show()

In [ ]:
train_data['target'].value_counts(normalize=True)


*Target variable is balanced*

In [ ]:
cat_cols = ['ProductName', 'EngineVersion','PlatformType', 'Processor', 'OSVersion',
       'OsPlatformSubRelease', 'SKUEditionName',
       'MDC2FormFactor', 'DeviceFamily', 'PrimaryDiskType', 'ChassisType',
       'PowerPlatformRole', 'OSArchitecture', 'OSBranch',
       'OSEdition', 'OSSkuFriendlyName', 'OSInstallType',
       'AutoUpdateOptionsName', 'OSGenuineState', 'LicenseActivationChannel',
       'FlightRing']
for col in cat_cols:
    plt.figure(figsize=(8,4))
    sns.countplot(y=train_data[col], order=train_data[col].value_counts().index)
    plt.title(f'Distribution of {col}')
    plt.show()


In [ ]:
num_cols = ['IsBetaUser', 'RealTimeProtectionState', 'IsPassiveModeEnabled',
       'AntivirusConfigID', 'NumAntivirusProductsInstalled',
       'NumAntivirusProductsEnabled', 'HasTpm', 'CountryID', 'CityID',
       'GeoRegionID', 'LocaleEnglishNameID', 'OSBuildNumber', 'OSProductSuite',
       'IsSystemProtected', 'AutoSampleSubmissionEnabled', 'SMode',
       'IEVersionID', 'FirewallEnabled', 'EnableLUA', 'OEMNameID',
       'OEMModelID', 'ProcessorCoreCount', 'ProcessorManufacturerID',
       'ProcessorModelID', 'PrimaryDiskCapacityMB', 'SystemVolumeCapacityMB',
       'HasOpticalDiskDrive', 'TotalPhysicalRAMMB',
       'PrimaryDisplayDiagonalInches', 'PrimaryDisplayResolutionHorizontal',
       'PrimaryDisplayResolutionVertical', 'InternalBatteryNumberOfCharges',
       'OSBuildNumberOnly', 'OSBuildRevisionOnly', 'OSInstallLanguageID',
       'OSUILocaleID', 'IsPortableOS', 'IsFlightsDisabled',
       'FirmwareManufacturerID', 'FirmwareVersionID', 'IsSecureBootEnabled',
       'IsVirtualDevice', 'IsTouchEnabled', 'IsPenCapable',
       'IsAlwaysOnAlwaysConnectedCapable', 'IsGamer', 'RegionIdentifier',
       'target']
fig, axes = plt.subplots(5, 5, figsize=(15, 12))  # Grid of subplots

for i, col in enumerate(num_cols[:25]):  # First 25 numerical columns
    row, col_index = divmod(i, 5)  # Compute row & column index
    axes[row, col_index].hist(train_data[col], bins=50, edgecolor='black', alpha=0.7)
    axes[row, col_index].set_title(col, fontsize=10)

fig.tight_layout(pad=2.0)  # Add spacing between subplots
plt.show()





In [ ]:
num_cols = train_data.select_dtypes(exclude = "object").columns
num_cols

In [ ]:
# Correlation of numerical features with target
correlation = train_data[num_cols].corr()['target'].sort_values(ascending=False)
print("\nFeature Correlation with Target:")
print(correlation)

*There is not Linear relationship between features and target columns, hence in that case linear models may not work*

In [ ]:
import pandas as pd
import numpy as np
  # Replace with your actual dataset

# Compute correlation matrix
correlation_matrix = train_data[num_cols].corr()

# Find the two columns with the highest absolute correlation (excluding self-correlation)
correlation_matrix = correlation_matrix.abs()
np.fill_diagonal(correlation_matrix.values, 0)  # Ignore self-correlation

# Get the highest correlated pair
max_corr = correlation_matrix.unstack().idxmax()
print(f"Highest correlation is between: {max_corr[0]} and {max_corr[1]}")
print(f"Correlation value: {correlation_matrix.loc[max_corr[0], max_corr[1]]}")

In [ ]:
# Convert Date Features to datetime format
train_data['DateAS'] = pd.to_datetime(train_data['DateAS'])
train_data['DateOS'] = pd.to_datetime(train_data['DateOS'])

test_data['DateAS'] = pd.to_datetime(test_data['DateAS'])
test_data['DateOS'] = pd.to_datetime(test_data['DateOS'])

# Extract useful date-based features
train_data['AS_Year'] = train_data['DateAS'].dt.year
train_data['OS_Year'] = train_data['DateOS'].dt.year
train_data['AS_Month'] = train_data['DateAS'].dt.month
train_data['OS_Month'] = train_data['DateOS'].dt.month
train_data['Days_Diff'] = (train_data['DateAS'] - train_data['DateOS']).dt.days

test_data['AS_Year'] = test_data['DateAS'].dt.year
test_data['OS_Year'] = test_data['DateOS'].dt.year
test_data['AS_Month'] = test_data['DateAS'].dt.month
test_data['OS_Month'] = test_data['DateOS'].dt.month
test_data['Days_Diff'] = (test_data['DateAS'] - test_data['DateOS']).dt.days

**1.There are many null values.
2. There are consant columns.
3. Have higher correlation between 2 columns.
4.There is no correlation between target and other numerical features.
5. Categorical columns contaion UNKNOWN values and numerical columns contains outliears.* **

## Handling Missing Values

In [ ]:
# Handling missing values
train_data.isna().sum().sum()

In [ ]:
test_data.isna().sum().sum()

In [ ]:
cat_col = train_data.select_dtypes(include = "object").columns
cat_col

In [ ]:

ID_col = ["CityID","IsVirtualDevice","SMode","FirewallEnabled","NumAntivirusProductsEnabled","NumAntivirusProductsInstalled","IsAlwaysOnAlwaysConnectedCapable","IsFlightsDisabled","IsGamer","EnableLUA","RealTimeProtectionState","FirmwareManufacturerID","FirmwareVersionID","OEMModelID","OEMNameID","OSInstallLanguageID","ProcessorModelID","ProcessorManufacturerID","AntivirusConfigID","IEVersionID"]

In [ ]:
cat_id_col = list(cat_col) + ID_col

In [ ]:
num_col = ["RegionIdentifier", "InternalBatteryNumberOfCharges", "IsSystemProtected",
                 "TotalPhysicalRAMMB", "PrimaryDiskCapacityMB", "SystemVolumeCapacityMB",
                 "ProcessorCoreCount", "PrimaryDisplayResolutionHorizontal", 
                 "PrimaryDisplayResolutionVertical", "PrimaryDisplayDiagonalInches"]

In [ ]:

# Define imputers
cat_id_imputer = SimpleImputer(strategy="most_frequent")  # Combined categorical + ID imputer
num_imputer = SimpleImputer(strategy="mean")  # Numerical imputer

# Column transformer to handle different imputations
preprocessor = ColumnTransformer([
    ("cat_id_impute", cat_id_imputer, cat_id_col),
    ("num_impute", num_imputer, num_col)
], remainder="passthrough") #what does this mean find out.

# Create pipeline
impute_pipeline = Pipeline([
    ("preprocess", preprocessor)
])

# Apply transformation
train_data[cat_id_col + num_col] = impute_pipeline.fit_transform(train_data[cat_id_col + num_col])
test_data[cat_id_col + num_col] = impute_pipeline.transform(test_data)

# Fill specific columns with fixed values
fill_values = {
    "DateOS": "2018-08-14", "OS_Year": "2018.0", "OS_Month": "8.0", "Days_Diff": "14.0",
    "DateAS": "2018-08-14", "AS_Year": "2018.0", "AS_Month": "8.0"
}

train_data.fillna(fill_values, inplace=True)
test_data.fillna(fill_values, inplace=True)


In [ ]:
train_data = train_data.drop(columns = ["DateOS","DateAS"])
test_data = test_data.drop(columns = ["DateOS","DateAS"])

In [ ]:
train_data.isna().sum().sum()

In [ ]:
test_data.isna().sum().sum()

In [ ]:
# Check for missing values
missing_values = test_data.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
print("\nMissing Values in Train Data:")
print(missing_values)

In [ ]:
test_data["GeoRegionID"] = test_data["GeoRegionID"].fillna(277.0)

In [ ]:
train_data.isna().sum().sum()

In [ ]:
test_data.isna().sum().sum()

In [ ]:
# Find constant columns (only 1 unique value)
constant_cols = [col for col in train_data.columns if train_data[col].nunique() == 1]

print("Constant Columns:", constant_cols)



In [ ]:
train_data = train_data.drop(columns =  ['IsBetaUser', 'AutoSampleSubmissionEnabled', 'IsFlightsDisabled'])
test_data = test_data.drop(columns =  ['IsBetaUser', 'AutoSampleSubmissionEnabled', 'IsFlightsDisabled'])


In [ ]:
num_cols

In [ ]:
cat_cols

# Columns Encoding

In [ ]:

# Define categorical columns for encoding
one_hot_cols = ["PlatformType", "Processor", "OSGenuineState", "OSArchitecture", "DeviceFamily", "OSInstallType", "OsPlatformSubRelease", "SKUEditionName", "PrimaryDiskType", "PowerPlatformRole", "AutoUpdateOptionsName", "LicenseActivationChannel", "FlightRing", "MDC2FormFactor", "OSBranch"]
count_enc_cols = ["EngineVersion", "AppVersion"]
target_enc_cols = ["SignatureVersion", "OSBuildLab", "NumericOSVersion", "OSEdition", "OSSkuFriendlyName", "ChassisType"]
ordinal_enc_cols = ["ProductName", "MachineID"]
os_version_col = "OSVersion"

# Define OS version mapping
os_order = ['6.1.1.0', '6.3.0.0', '10.0.0.0', '10.0.1.0', '10.0.2.0', '10.0.3.0', '10.0.4.0']
os_mapping = {version: idx for idx, version in enumerate(os_order)}

# Define transformers
one_hot_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
count_encoder1 = CountEncoder()
target_encoder1 = TargetEncoder()
ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("onehot", one_hot_encoder, one_hot_cols),
        ("count_enc_1", count_encoder1, count_enc_cols),
        ("target_enc_1", target_encoder1, target_enc_cols),
        ("ordinal_enc",ordinal_encoder,ordinal_enc_cols)
    ],
    remainder="passthrough"
)

# Complete pipeline
pipeline = Pipeline([
    ("preprocessor", preprocessor)
])

# Apply pipeline to train and test data
# Apply pipeline to train and test data (excluding target column)
train_features = train_data.drop(columns=['target'])  # Drop target for transformation
train_D = pipeline.fit_transform(train_features, train_data['target'])
test_D = pipeline.transform(test_data)


# Convert transformed NumPy array back to DataFrame
train_D = pd.DataFrame(train_D, columns=pipeline.get_feature_names_out())
test_D = pd.DataFrame(test_D, columns=pipeline.get_feature_names_out())

# Convert OSVersion using mapping
train_D["OSVersion_encoded"] = train_data["OSVersion"].map(os_mapping).values
test_D["OSVersion_encoded"] = test_data["OSVersion"].map(os_mapping).values

train_D = train_D.drop(columns = "remainder__OSVersion")
test_D = test_D.drop(columns = "remainder__OSVersion")

In [ ]:

train_D["remainder__OS_Year"] = train_D["remainder__OS_Year"].astype(float)
train_D["remainder__OS_Month"] = train_D["remainder__OS_Month"].astype(float)
train_D["remainder__Days_Diff"] = train_D["remainder__Days_Diff"].astype(float)  # Use int if all values are whole numbers

test_D["remainder__AS_Year"] = test_D["remainder__AS_Year"].astype(float)
test_D["remainder__AS_Month"] = test_D["remainder__AS_Month"].astype(float)
#test_D["remainder__OS_Year"] = test_D["remainder__OS_Year"].astype(float)  # Use int if all values are whole numbers

In [ ]:
# Convert all object columns that contain numbers into numeric types
for col in train_D.select_dtypes(include=['object']).columns:
    train_D[col] = pd.to_numeric(train_D[col], errors='coerce') # Converts to float if needed
for col in test_D.select_dtypes(include=['object']).columns:
    test_D[col] = pd.to_numeric(test_D[col], errors='coerce')


In [ ]:
train_D.select_dtypes(include  = "object").columns

## Feature Selection and Scaling

In [ ]:
X = train_D
Y = train_data["target"]

**Scaling Features**

In [ ]:
# Standardize Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
#X_test_scaled = scaler.transform(test_data)
X_test_scaled = scaler.transform(test_D)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=test_D.columns)


**Feature Selection**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train_scaled, Y)
feature_importance = pd.Series(model.feature_importances_, index=X.columns)
important_features = feature_importance[feature_importance > 0.01].index
train_selected2 = X_train_scaled[important_features]
test_selected2 =  X_test_scaled[important_features]


In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
selector = SelectKBest(mutual_info_classif, k=10)  # Select top 10 features
train_selected3 = selector.fit_transform(X_train_scaled, Y)
test_selected3 = selector.transform(X_test_scaled)



In [ ]:
# Train XGBClassifier
xgb = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
xgb.fit(X_train, y_train)

# Select important features
selector = SelectFromModel(xgb, prefit=True)
train_selected4 = selector.transform(X_train_scaled)
test_selected4 = selector.transform(X_test_scaled)

In [ ]:
train_selected4.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Split Data
X_train, X_test, Y_train, Y_test = train_test_split(train_selected2, Y, test_size=0.2, random_state=42)

# Train Model
model1 = RandomForestClassifier()
model1.fit(X_train, Y_train)

from sklearn.metrics import mean_squared_error
y_val_pred = model1.predict(X_test)
# Evaluate model performance
mse = mean_squared_error(Y_test, y_val_pred)
print(f"Validation MSE: {mse}")
y_pred1 = model1.predict(test_selected2)

In [ ]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, Y_train, Y_test = train_test_split(train_selected3, Y, test_size=0.2, random_state=42)
model2 = LogisticRegression()
model2.fit(X_train, Y_train)

y_val_pred = model2.predict(X_test)
mse = mean_squared_error(Y_test, y_val_pred)
print(f"Validation MSE: {mse}")

# ✅ Step 4: Predict Values
y_pred2 = model2.predict(test_selected3)



In [ ]:
from xgboost import XGBClassifier 
# Train XGBoost model
X_train, X_test, Y_train, Y_test = train_test_split(train_selected4, Y, test_size=0.2, random_state=42)
model3 = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model3.fit(X_train, Y_train)

y_val_pred = model3.predict(X_test)
mse = mean_squared_error(Y_test, y_val_pred)
print(f"Validation MSE: {mse}")
# Predictions
y_pred3 = model3.predict(test_selected4)




In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

x_train,x_test,y_train,y_test = train_test_split(train_selected4,Y,test_size = 0.2)

param_dist = {
    'max_depth': np.random.randint(3, 10, 5),
    'learning_rate': np.linspace(0.01, 0.2, 10),
    'n_estimators': np.random.randint(100, 500, 10)
}

random_search = RandomizedSearchCV(XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
                                   param_dist, n_iter=10, scoring='accuracy', cv=5, verbose=2)
random_search.fit(train_selected4,Y)

print(random_search.best_params_)
best_xgbclassifier_model = random_search.best_estimator_

In [ ]:
y_pred4 = best_xgbclassifier_model.predict(test_selected4)

In [ ]:
Y_test = submission_data["target"].values  # Extract target values
Y_test

In [ ]:
Y_test.shape

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

acc1 = print("RandomClassifier Accuracy:",accuracy_score(Y_test,y_pred1))
acc2 = print("Logistic Accuracy:",accuracy_score(Y_test,y_pred2))
acc3 = print("XGBClassifier:",accuracy_score(Y_test,y_pred3))
acc4 = print("Best_XGBClassifier:",accuracy_score(Y_test,y_pred4))
print(classification_report(Y_test,y_pred4))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,y_pred4)

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Pred 0', 'Pred 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel("Predicted Label")
plt.ylabel("Actual Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

print("Precision Score:",precision_score(Y_test,y_pred4))
print("Recall Score:",recall_score(Y_test,y_pred4))
print("f1 Score:",f1_score(Y_test,y_pred4))

In [ ]:
submission = pd.DataFrame({'id': range(0,test_data.shape[0]),'target':y_pred4})
submission.to_csv('submission.csv',index = False)

In [ ]:
submission